In [2]:
import numpy as np
import scipy.linalg

In [8]:
def ldl_rank_one_update(l, d, alpha, w):
    '''
    l: nxn matrix
    d: nx1 vector
    alpha: scalar
    w: nx1 vector
    '''
    w = np.copy(w)
    size = len(w)

    assert(l.shape[0] == size)
    assert(l.shape[1] == size)
    assert(len(d) == size)

    l_bar = np.zeros((size, size))
    d_bar = np.zeros(size)
    beta = np.zeros(size)

    for j in range(0, size):
        if not np.allclose(w[j], 0.0):
            if not np.allclose(d[j], 0.0):
                p = w[j]
                d_bar[j] = d[j] + alpha * (p ** 2.0)
                beta[j] = p * alpha / d_bar[j]
                alpha = d[j] * alpha / d_bar[j]

                for r in range(j, size):
                    w[r] = w[r] - p * l[r, j]
                    l_bar[r, j] = l[r, j] + beta[j] * w[r]
            else:
                p = w[j]
                d_bar[j] = alpha * (p ** 2.0)
                beta[j] = 1.0 / p
                for r in range(j, size):
                    l_bar[r, j] = beta[j] * w[r]
                for i in range(j + 1, size):
                    d_bar[i] = d[i]
                    for r in range(i, size):
                        l_bar[r, i] = l[r, i]

                break
        else:
            d_bar[j] = d[j]
            alpha = alpha
            for r in range(j, size):
                w[r] = w[r]
                l_bar[r, j] = l[r, j]

    #print("w =", w)
    #print("beta =", beta)
    #print("l_bar =\n", l_bar)
    #print("d_bar =", d_bar)

    return l_bar, d_bar


def forward_substitution(l, b):
    assert(l.shape[0] == l.shape[1])
    assert(l.shape[0] == b.shape[0])

    n = l.shape[0]
    x = np.zeros(n)

    for i in range(0, n):
        x[i] = b[i]
        for j in range(0, i):
            x[i] -= l[i, j] * x[j]
        x[i] /= l[i, i]

    return x

def backward_substitution(u, b):
    assert(u.shape[0] == u.shape[1])
    assert(u.shape[0] == b.shape[0])

    n = u.shape[0]
    x = np.zeros(n)

    for i in range(n - 1, -1, -1):
        x[i] = b[i]
        for j in range(i + 1, n):
            x[i] -= u[j, i] * x[j]
        x[i] /= u[i, i]

    return x

def ldl_solve(l, d, b):
    '''
    Solve A * x = b via LDL decomposition. The decomposition of A must be given
    by the parameters l and d so that A = L * D * L^T
    '''
    # Solve L * y = b for y
    y = forward_substitution(l, b)


    # Solve D * L^T * x = y for x

    # First step: z = D^{-1} y
    #z = np.divide(y, d)

    n = len(d)
    z = np.zeros(n)
    for i in range(0, n):
        # This check is a _choice_ we could, e.g. also
        # 1. ignore the problem and get NaN's or Inf's
        # 2. monitor and return a "trouble" code
        # 3. throw an exception (closely linked to the previous point)
        # 4. Accept a parameter that "masks out" singular values (i.e.
        #    explicitly reduces the system of equations)
        # The _physical_ interpretation of the below approaches is
        # difficult/non-existant

        tau = 0.0

        # 1. "Truncated SVD" (note though that this is neither SVD, nor truncation of all following values, but only ignoring one)
        if not np.allclose(d[i], 0.0):
            tau = 1.0 / d[i]

        # 2. "Damped least squares" (if LDL is used to solve a LSQ problem)
        # see: Samuel R. Buss, Introduction to Inverse Kinematics with Jacobian Transpose, Pseudoinverse and Damped Least Squares methods, 2009.
        lambda_damping = 0.001
        tau = d[i] / (d[i] ** 2 + lambda_damping ** 2)

        # 3. "Selectively damped least squares" (if LDL is used to solve a LSQ problem)
        # see: https://github.com/orocos/orocos_kinematics_dynamics/blob/master/orocos_kdl/src/chainiksolvervel_wdls.cpp#L174
        # ...

        # 4. What about _weighted_ least squared?


        z[i] = tau * y[i]

    # Second step: solve L^T * x = z for x
    x = backward_substitution(l, z)

    return x


a = np.random.rand(6, 6)
#A = np.identity(6)
A = np.zeros((6, 6))
#A = np.dot(a.transpose(), a)
# w = np.array(h)
w = np.random.rand(6)
print("W",w)
alpha = 1.0

print("A =\n", A)
#_, s, _ = np.linalg.svd(A)
#print("svd(A) =\n", s)


# Initial LDL decomposition
l_init, d_init, perm_init = scipy.linalg.ldl(A)
l_init = l_init[perm_init, :]
#d_init = np.diag(d_init)[perm_init]
d_init = np.diag(d_init)
print("l_init", l_init)
print(perm_init)
print("d_init",d_init)



#l_init = np.array([[1.0, 5.0],
#                   [5.0, 1.0]])
#d_init = np.array([2.0, 3.0])
#alpha = 1.0 / 4.0
#w = np.array([1.0, 2.0])

#l_init = np.array([[1.0, 0.0, 0.0],
#                    [2.0, 1.0, 0.0],
#                    [3.0, 4.0, 1.0]])
# d_init = np.array([1.0, 2.0, 3.0])
# alpha = 0.5
# w = np.array([2.0, 3.0, 4.0])

# rank_one(ldl(A))
lp, dp = ldl_rank_one_update(l_init, d_init, alpha, w)
print("\n\nrank_one(ldl(A)):")
print(lp)
print(dp)


# ldl(rank_one(A))
Ap = np.add(A, alpha * np.outer(w, w))
lu, d, perm = scipy.linalg.ldl(Ap)
lu = lu[perm, :]
#d = np.diag(d)[perm]
d = np.diag(d)
print("\n\nldl(rank_one(A)):")
print(lu)
print(perm)
print(d)



print(np.allclose(lp, lu))
print(np.allclose(dp, d))






b = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
x = ldl_solve(lp, dp, b)
print("solve(rank_one(ldl(A)), b) =", x[perm_init])

x = ldl_solve(lu, d, b)
print("solve(ldl(rank_one(A)), b) =", x)

#x = np.linalg.solve(Ap, b)
#print(x)



print("Ap:\n", Ap)
print("pinv(A) * b = ", np.dot(np.linalg.pinv(Ap), b))















# LDL decomposition of diagonal matrix (L seems to be diagonal, too)
a = np.random.rand(3, 3)
b = np.random.rand(3, 3)
A = np.dot(a.transpose(), a)
B = np.dot(b.transpose(), b)
L = np.block([[A, np.zeros((3, 3))], [np.zeros((3, 3)), B]])
scipy.linalg.ldl(L)


W [0.89531959 0.73008421 0.49943387 0.1329639  0.42805911 0.14680344]
A =
 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
l_init [[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
[0 1 2 3 4 5]
d_init [0. 0. 0. 0. 0. 0.]


rank_one(ldl(A)):
[[1.         0.         0.         0.         0.         0.        ]
 [0.81544536 1.         0.         0.         0.         0.        ]
 [0.55782748 0.         1.         0.         0.         0.        ]
 [0.14850998 0.         0.         1.         0.         0.        ]
 [0.47810761 0.         0.         0.         1.         0.        ]
 [0.16396764 0.         0.         0.         0.         1.        ]]
[0.80159717 0.         0.         0.         0.         0.        ]


ldl(rank_one(A)):
[[ 1.          0.          0.          0.          0.          0.        ]
 [ 0.81544536  1.        

(array([[1.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.95411872, 1.        , 0.        , 0.        , 0.        ,
         0.        ],
        [1.10885085, 1.1516775 , 1.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 1.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 1.13646789, 1.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.72907795, 0.12724686,
         1.        ]]),
 array([[1.09451639, 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.18862226, 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.18370271, 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.72822155, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.55373637,
         0.        